# Foreword

In this notebook we show how to use the Talos tool for automatic network optimization (and, in general, hyperparameter tuning). This example uses the breast cancer dataset which we have already seen in the course and is completely self contained. However if you want to further understand what's going on please refer to:

* [Talos library official website](https://pypi.org/project/talos/)
* [Talos github repository, with examples](https://github.com/autonomio/talos)
* [Talos documentation](https://autonomio.github.io/talos/#/README?id=quick-start)

# Setup(s)

## Standard libraries setup

In [4]:
#very common libraries, that we for sure are using
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## Talos setup

In [1]:
#making sure talos is installed. As of Oct 2024, talos requires tensorflow 2.14,
#which is older than the default 2.17 installad on Colab. As a result after running
#this code block you need to restart the notebook (not delete, just restart)
!pip install talos

#importing the library
import talos

## Seed setup

In [2]:
#resetting the seeds
!wget -O support_code.py https://raw.githubusercontent.com/ne1s0n/coding_excercises/master/lab_day1/support_code.py
%run support_code.py
n1 = 0
reset_random_seeds(n1)

--2024-10-02 08:56:52--  https://raw.githubusercontent.com/ne1s0n/coding_excercises/master/lab_day1/support_code.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6242 (6.1K) [text/plain]
Saving to: ‘support_code.py’

support_code.py     100%[===================>]   6.10K  --.-KB/s    in 0s      

2024-10-02 08:56:52 (70.4 MB/s) - ‘support_code.py’ saved [6242/6242]

Import all libraries: yes
importing libraries
Defining functions
DONE!


## Data setup

In [5]:
#libraries for this block
import sklearn.datasets
from sklearn.model_selection import StratifiedShuffleSplit

# loading data
from sklearn.datasets import load_breast_cancer
bcancer = load_breast_cancer()
y = bcancer.target
X = pd.DataFrame(bcancer.data, columns=bcancer.feature_names)

# normalizing
X = (X - X.mean())/X.std()

# Talos workflow

## Hyperparamaters to be explored

This part becomes central. We define the space (i.e. the amount of combinations) that we are going to explore.

In [6]:
#talos requires a dictionary
my_parameters = {
    #these hyperparameters need to be optimized
    'first_layer': [12, 24, 48],
    'second_layer': [12, 24, 48],
    'LR' : [0.0001, 0.1, 10]
}

## Train function (default, without Talos)

We report below a basic function used to train and return a model. It doesn not accept any hyperparameter (all values are hard coded). It will not be used in the example, but it serves as reference for when we integrate Talos.

In [7]:
#libraries for this block
from keras.models import Sequential
from keras.layers import Dense, Input
from tensorflow.keras.optimizers import SGD

# a function to declare and train the network. Returns the trained model
def train_net_default(x_train, y_train, x_val, y_val):

  #this depends on the input data
  input_shape = (x_train.shape[1],)

  #a simple neural network with two hidden layers
  model = Sequential()
  model.add(Input(input_shape))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(5, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  #instantiating the optimizer, compiling, training
  opt = SGD(learning_rate=0.1)
  model.compile(optimizer=opt, loss='binary_crossentropy')
  history = model.fit(x=x_train, y=y_train, validation_data=[x_val, y_val],
                      epochs=100, verbose=0)

  return(model)


## Train function, with Talos

In [8]:
# a function to declare and train the network, accepting a dictionary of
# hyperparameters. It must return both the trained model and the training history
def train_net_talos(x_train, y_train, x_val, y_val, par):

  #this depends on the input data
  input_shape = (X.shape[1],)

  #a simple neural network with two hidden layers
  model = Sequential()
  model.add(Input(input_shape))
  model.add(Dense(par['first_layer'], activation='relu'))
  model.add(Dense(par['second_layer'], activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  #instantiating the optimizer, compiling, training
  opt = SGD(learning_rate=par['LR'])
  model.compile(optimizer=opt, loss='binary_crossentropy')
  history = model.fit(x=x_train, y=y_train, validation_data=[x_val, y_val],
                      epochs=100, verbose=0)

  #returning both history and model, in that order
  return(history, model)

## Running a Talos "scan"

In this simple example Talos will check all the available combinations, once.

In [9]:
#note: Talos does not directly support Pandas dataframes. However, it's quite
#straightforward to obtain a table using .values
t = talos.Scan(x=X.values, y=y, params=my_parameters, model=train_net_talos, experiment_name='breast_cancer')

100%|██████████| 27/27 [03:11<00:00,  7.10s/it]


We just obtained a Scan object, which contain in the `.data` field information on all the tested combinations.

In [10]:
print(type(t))
print(t.data.shape)

<class 'talos.scan.Scan.Scan'>
(27, 9)


Taking a look at the results

In [11]:
t.data

,start,end,duration,round_epochs,loss,val_loss,first_layer,second_layer,LR
0,10/02/24-085840,10/02/24-085847,6.731426,100,0.629216,6.265510e-01,12,12,0.0001
1,10/02/24-085847,10/02/24-085853,5.671406,100,0.005924,1.764257e-01,12,12,0.1000
2,10/02/24-085853,10/02/24-085859,5.379792,100,0.851921,5.956609e+11,12,12,10.0000
3,10/02/24-085859,10/02/24-085905,5.583386,100,0.609602,6.185047e-01,12,24,0.0001
4,10/02/24-085905,10/02/24-085911,5.820507,100,0.005689,2.193291e-01,12,24,0.1000
5,10/02/24-085911,10/02/24-085917,5.582974,100,0.317384,3.263626e-01,12,24,10.0000
6,10/02/24-085917,10/02/24-085923,5.921209,100,0.561008,5.674707e-01,12,48,0.0001
7,10/02/24-085923,10/02/24-085929,5.597608,100,0.009588,1.391487e-01,12,48,0.1000
8,10/02/24-085929,10/02/24-085940,10.920617,100,0.839853,5.971889e+04,12,48,10.0000
9,10/02/24-085940,10/02/24-085951,10.748406,100,0.668380,6.242418e-01,24,12,0.0001


Extracting the best configuration

In [12]:
#a local copy, for easier notation
df = t.data

#printing the row with lowest validation loss
df[df.val_loss == df.val_loss.min()]

,start,end,duration,round_epochs,loss,val_loss,first_layer,second_layer,LR
7,10/02/24-085923,10/02/24-085929,5.597608,100,0.009588,0.139149,12,48,0.1


Extracting the best performing model

In [13]:
#I need to specify what is the criterion (i.e. the metric) used to define the "best" model.
#Moreover, "asc" has to be True for the case where the metric is something to be minimized.
best_model = t.best_model(metric='val_loss', asc=True)
print(type(best_model))

<class 'keras.src.engine.sequential.Sequential'>


# Further steps

The code above is a **very** minimal example and works as a starting point. Stuff to consider:

* each combination of hyperparameter is trained once, with a 70/30 default split. Using `.evaluate_models()` it's possible to do a proper k-fold crossvalidation (see [scan documentation](https://autonomio.github.io/talos/#/Scan), search "evaluate_models")
* the default approach of trying all the combinations can become unfeasible very quickly. The `Scan` function supports several policies for sampling a subset of the hyperparameter space. See the [Towardsdatascience's tutorial](https://towardsdatascience.com/tune-the-hyperparameters-of-your-deep-learning-networks-in-python-using-keras-and-talos-2a2a38c5ac31) for a more in-depth example